# Checkout Flow Class Design

## Listing

Listing is an action to make a product to be sell in Tokopedia
To list the product, inputs needed are product name (prod_name), product price (prod_price), store location (store_loc), stock, weight.
Here are some rules in listing product:
- Product price must be at least in hundred rupiahs. If it is not, then the price will be rounded to the nearest hundred rupiahs.
- Store location must be in "Sumatera", "Jawa", "Kalimantan", "Sulawesi", or "Papua". If seller list a product with another store location, it will be counted as invalid
- Stock must be 0 or more
- Weight must be more than 0 (weight is in kg)

There are 6 method for a listing class:
1. Update stock : it is to change the stock by seller. If new stock is negative. The stock will not be changed
2. Update price : it is to change the price by seller. Price still have rule explained above.
3. Update store location : it is to change the location by seller. If new location doesn't follow the rule above. It won't be changed.
4. Update weight : it is to change the weight by seller. If new weight doesn't follow the rule above. It won't be changed.
5. Set discount

#### 1. Import Library

In [121]:
import datetime

#### 2. Initialization Class Listing

In [205]:
class Listing:
    def __init__(self, prod_name, prod_price, store_loc, stock, weight):
        self.prod_name = prod_name
        #PRODUCT PRICE
        if prod_price%100!=0:
            print("Product price invalid. the price will be rounded to hundred rupiahs")
            self.prod_price = round((prod_price/100))*100
        else:
            self.prod_price = prod_price
        #STORE LOCATION
        if store_loc not in ["Sumatera", "Jawa", "Kalimantan", "Sulawesi", "Papua"]:
            print("Please choose valid location (\"Jawa\", \"Sumatera\", \"Kalimantan\", \"Sulawesi\" or \"Papua\")")
            self.store_loc = "Invalid"
        else:
            self.store_loc = store_loc
        #STOCK
        if stock<0:
            print("Stock must be 0 or more")
            self.stock = 0
        else:
            self.stock = stock
        #WEIGHT (kg)
        if weight<=0:
            print("Please insert a valid weight (more than 0).")
            self.weight = 1
        else:
            self.weight = weight
        self.discount = 0
        self.selling_price = self.prod_price-self.discount
        self.headline = f"We sell {prod_name} at {self.selling_price} IDR today!"
        
    def update_stock(self, new_stock):
        if self.stock + new_stock < 0:
            print("Stock can't go below zero. The stock will not be changed")
        else:
            self.stock += new_stock
    
    def update_price(self, new_price):
        if new_price % 100 != 0:
            print("Product price invalid. The price will be rounded to hundred rupiahs")
            self.prod_price = round((new_price/100))*100
        else:
            self.prod_price = new_price
        
    def update_store_loc(self, new_loc):
        if new_loc not in ["Sumatera", "Jawa", "Kalimantan", "Sulawesi", "Papua"]:
            print("Please choose valid location (\"Jawa\", \"Sumatera\", \"Kalimantan\", \"Sulawesi\" or \"Papua\"). \n\
                 Store location will not be changed.")
        else:
            self.store_loc = new_loc
    
    def update_weight(self, new_weight):
        if weight<=0:
            print("Please insert a valid weight (more than 0). Weight will not be changed.")
        else:
            self.weight = new_weight
    
    def set_discount(self, discount):
        if discount>0 and discount>=self.prod_price:
            if discount%100!=0:
                print("Discount invalid. The discount will be rounded to hundred rupiahs")
                self.discount = round((prod_price/100))*100
            else:
                self.discount = discount
        else :
            if discount<=0:
                print("Discount can not be 0 or less")
            elif discount>=self.prod_price:
                print("Discount can not more than normal price")
    
    def shipping_cost(self, location, service, weight = 1): #service can be "Reguler" or "Sameday"
        ourLoc = {"Sumatera":1, "Jawa":2, "Kalimantan":3, "Sulawesi":4, "Papua":5}
        ourRegulerCost = [10000, 20000, 30000, 40000, 50000]
        if location not in ["Sumatera", "Jawa", "Kalimantan", "Sulawesi", "Papua"]:
            print("Please choose valid location (\"Jawa\", \"Sumatera\", \"Kalimantan\", \"Sulawesi\" or \"Papua\")")
        elif service not in ["Reguler", "Sameday"]:
            print("Please choose on of our services (\"Reguler\" or \"Sameday\")")
        else:
            if self.store_loc!=location:
                if service=="Sameday":
                    print("Same day delivery can't be done in this location. Please choose another service (\"Reguler\")")
                else:
                    return ourRegulerCost[abs(ourLoc[self.store_loc]-ourLoc[location])]*weight
            else:
                if service=="Sameday":
                    return 25000*weight
                else:
                    return ourRegulerCost[0]*weight
    
    #def add_to_catalog(self, catalog):
        


#### 3. Initialization Class ShoppingCart

This class initialized with:
- `items` : a list of Listing that shows item in cart
- `total_price` : total price of the current items in cart
- `checkout_time` : save the check out time
- 

In [396]:
class ShoppingCart():
    def __init__(self, buyer_loc, service):
        self.items = []
        self.total_price = 0
        self.checkout_time = 0
        self.buyer_loc = buyer_loc
        self.service = service
        self.total_shipping_price = 0
        self.item_counter = 0
        self.product_number = {}
        
    def view_item(self):
        print("Your cart items :")
        for i in range(0, len(self.items)):
            if self.items[i][1] > 0:
                print("*", self.items[i][0].prod_name, " : ", self.items[i][1])
        print("Price: ", self.total_price, "\nShipping price: ", self.get_shipping_price())
    
    def add_item(self, product, qty):
        if qty <= product.stock:
            if product.prod_name in self.product_number:
                self.items[self.product_number[product.prod_name]][1] += qty
                self.total_price += (product.prod_price * qty)
                product.update_stock(-qty)
            else:
                self.product_number[product.prod_name] = self.item_counter
                self.item_counter += 1
                product.update_stock(-qty)
                self.items.append([product, qty])
                self.total_price += (product.prod_price * qty)
        else:
            print("Your quantity is exceeding the product stock")
        
    def remove_item(self, product):
        self.total_price -= (product.prod_price * self.items[self.product_number[product.prod_name]][1])
        product.update_stock(-self.items[self.product_number[product.prod_name]][1])
        self.items[self.product_number[product.prod_name]][1] = 0
    
    def update_qty(self, product, up_qty):
        for i in range(len(self.items)):
            if self.items[i][0] == product:
                if self.items[i][1] + up_qty > 0:
                    self.items[i][1] += up_qty
                    self.total_price += (up_qty * product.prod_price)
                else:
                    print("Quantity can't go zero or below, try to remove item instead")
    
    def get_shipping_price(self):
        total_shipping_price = 0
        for i in range(0, len(self.items)):
            total_shipping_price += self.items[i][0].shipping_cost(self.buyer_loc, self.service, self.items[i][0].weight * self.items[i][1])
        return total_shipping_price
    
    def get_total_price(self):
        self.total_shipping_price = self.get_shipping_price()
        self.total_price += (self.total_shipping_price)
        return self.total_price
    
    def checkout_now(self):
        self.checkout_time = datetime.datetime.now()
        print(f"----------------------------\nSuccess, you have checkout at {self.checkout_time}. \nShipping price: {self.get_shipping_price()} \nTotal price: {self.get_total_price()}")
        

### 4. Listing Process
#### 4.1. Listing items into marketplace

In [397]:
vacum = Listing("Vacum", 200000, "Papua", 100, 20)
sepeda = Listing("Sepeda", 2800000, "Jawa", 100, 60)
ransel = Listing("Ransel", 1000000, "Sumatera", 1000, 1)

#### 4.2. Checking shipping cost

In [398]:
{vacum.shipping_cost("Papua", "Sameday", ),
sepeda.shipping_cost("Sumatera", "Reguler")}

{20000, 25000}

### 5. Shopping Process
#### 5.1. Build a cart

In [399]:
myCart = ShoppingCart("Jawa", "Reguler")

#### 5.2. Checking items in bucket

In [400]:
myCart.view_item()

Your cart items :
Price:  0 
Shipping price:  0


#### 5.3. Add items into bucket

In [401]:
myCart.add_item(vacum, 1)
myCart.add_item(sepeda, 5)
myCart.add_item(ransel, 10)

#### 5.4. Checking items in bucket after add items

In [402]:
myCart.view_item()

Your cart items :
* Vacum  :  1
* Sepeda  :  5
* Ransel  :  10
Price:  24200000 
Shipping price:  4000000


#### 5.4. Updating quantity of items

In [403]:
myCart.update_qty(vacum, 10)
myCart.update_qty(ransel, -10)
myCart.update_qty(ransel, -9)

Quantity can't go zero or below, try to remove item instead


#### 5.5. Checking items in bucket

In [404]:
myCart.view_item()

Your cart items :
* Vacum  :  11
* Sepeda  :  5
* Ransel  :  1
Price:  17200000 
Shipping price:  11820000


#### 5.6. Remove item from bucket


In [407]:
myCart.remove_item(vacum)

myCart.view_item()

Your cart items :
* Sepeda  :  5
* Ransel  :  1
Price:  18020000 
Shipping price:  3020000


#### 5.7. Checkout

In [408]:
myCart.checkout_now()

----------------------------
Succes, you have checkout at 2019-07-23 08:57:22.418387. 
Shipping price: 3020000 
Total price: 21040000


In [8]:
print("Same day delivery can't be done in this location. \
Please choose another service (\"Reguler\" or \"Express\")")

Same day delivery can't be done in this location. Please choose another service ("Reguler" or "Express")
